# Operações de Simulação com o Servidor ASA

Este notebook detalha operações essenciais realizadas em simulações com o servidor ASA, incluindo autenticação, manipulação de simulações e aliases, criação de Design of Experiments (DOE), listagem de métricas, execução de lotes e recuperação de lotes.

Índice:
1. [Login e Autenticação](#login)
2. [Carregando e Salvando Simulações](#carregando_simulacoes)
3. [Listagem de Aliases](#listagem_aliases)
4. [Edição dos Aliases](#edicao_aliases)
5. [Criando Design de Experimentos (DOE)](#criando_doe)
6. [Listando Métricas](#listando_metricas)
7. [Executando um Lote com Parada Automática e Barra de Progresso](#executando_lote)
8. [Recuperando um Lote de Execuções](#recuperando_lote)

Esta célula importa todos os módulos necessários para execução e análise das simulações.

In [ ]:
import asapy  # Fornece vinculações Python para a API AsaPy
import asaclient # O módulo 'asaclient' é usado para estabelecer uma interface de cliente Python com o servidor Asa.

## Login e Autenticação <a name="login"></a>

A primeira etapa envolve login e autenticação, necessários para o usuário acessar o sistema. Inicializamos uma instância da classe `ASA` do módulo `asaclient`, definindo o parâmetro do servidor para a URL especificada. Esta ação estabelece uma conexão com o Servidor ASA.

In [ ]:
asa = asaclient.ASA(server="https://service.asa.dcta.mil.br")


## Carregando e Salvando Simulações <a name="carregando_simulacoes"></a>

Nesta seção, carregamos manualmente uma simulação a partir de um arquivo JSON. Este processo envolve abrir o arquivo JSON, carregar seu conteúdo em um dicionário Python e usar esses dados para instanciar um objeto de simulação.

In [ ]:
simulation_name = "nav_demo"
sim = asapy.load_simulation(f"./simulations/nav_demo.json")  # Creating a Simulation from the JSON file at the specified path, and assigning it to 'sim'.
sim = asa.save_simulation(sim)
sim

## Listagem de Aliases <a name="listagem_aliases"></a>

Esta seção envolve a enumeração de todos os aliases presentes em uma simulação.

Primeiro instanciamos um objeto da classe ``Doe`` (Design of Experiments). Em seguida, processamos os aliases (parâmetros utilizados nas simulações) para a simulação escolhida ``sim``, usando o método ``process_aliases_by_sim`` e passando as configurações dos componentes do servidor ASA. Os aliases processados são armazenados na variável ``aliases`` e exibidos. A função ``asa.component_configs()`` retorna uma lista de configurações de componentes. Essa informação é crucial para entender os números padrão, mínimo e máximo de componentes em um determinado cenário.

In [ ]:
doe = asapy.Doe()
aliases = doe.process_aliases_by_sim(sim, asa.component_configs()) 
aliases

## Edição dos Aliases <a name="edicao_aliases"></a>

Esta seção permite que os usuários modifiquem os parâmetros dos aliases listados para atender às suas necessidades específicas.

Esta operação ilustra como você pode ajustar manualmente o intervalo de um parâmetro de alias dentro do seu Design de Experimentos. Após o ajuste, o DataFrame 'aliases' atualizado é exibido.

In [ ]:
aliases.loc['hold_speed_kt','min'] = 100
aliases.loc['hold_speed_kt','max'] = 500
aliases.loc['hold_altitude_ft','min'] = 1000
aliases.loc['hold_altitude_ft','max'] = 10000
aliases

## Criando Design de Experimentos (DOE) <a name="criando_doe"></a>

Os usuários têm a capacidade de estabelecer um DOE, que lhes permite planejar, conduzir e analisar um conjunto de experimentos de maneira sistemática.

Nesta célula, estamos usando o método 'create' do objeto 'Doe' para gerar uma tabela de Design de Experimentos (DOE). O método recebe como argumentos os aliases processados anteriormente e o número de amostras. O DOE resultante é um DataFrame do pandas, que é armazenado na variável 'df' e então exibido.
O DOE é criado usando um método de Amostragem de Hipercubo Latino (LHS) e um tamanho de amostra samples. A função retorna um novo DataFrame com os nomes e valores das variáveis de entrada de acordo com o seu tipo.

In [ ]:
df = doe.create(aliases, samples=1000, seed=101)
df

## Listando Métricas <a name="listando_metricas"></a>

Esta etapa envolve a geração de uma lista de todas as métricas usadas em uma simulação.

O seguinte segmento de código recupera e processa as métricas associadas à simulação ``sim``. O método ``process_metrics`` do objeto ``Doe`` é usado para obter um DataFrame, ``metrics_df``, contendo os dados das métricas. A linha de código subsequente seleciona a linha rotulada como ``Team_Metrics_Blue`` do DataFrame e recupera as métricas correspondentes usando a coluna ``metrics``.

In [ ]:
metrics_df = doe.process_metrics(sim)
metrics_df

Inspecionando especificando as métricas do time `blue`

In [ ]:
print(metrics_df.loc['Team_Metrics_Blue']['metrics'])

A variável `metric` é atribuída ao valor da string `acft_standing`, representando uma métrica específica de interesse no contexto da análise. Esta métrica será importante ao implementar o recurso de parada automática durante a execução em lote.


In [ ]:
metric = 'acft_standing' 
side= 'blue'

## Executando um Lote com Parada Automática e Barra de Progresso <a name="executando_lote"></a>

Este recurso permite aos usuários executar um processo em lote e uma barra de progresso para acompanhar visualmente sua conclusão.

O lote está associado à simulação identificada por ``sim.id``. Após sua criação, ele é enviado para o sistema para posterior processamento e eventual execução.

Os códigos a seguir inicializa um objeto `ExecutionController` chamado `ec`. O parâmetro `sim_func` é definido como `asapy.batch_simulate(batch=batch)`, que especifica a função responsável pela execução da simulação em lote.

`asa::recorder::AsaMonitorReport` traz os relatórios de monitoramento para a simulação, ou seja, as métricas utilizadas para controlar a execução do lote estão listadas.

O parâmetro `stop_func` é definido como `asapy.stop_func(metric=metric, threshold=0.001)`, que indica a função a ser usada para parar a execução com base em uma métrica e um limite especificados.

O parâmetro `chunk_size` determina o tamanho de cada pedaço de simulações a ser executado simultaneamente.

In [ ]:
batch = asaclient.Batch(label=simulation_name, simulation_id=sim.id)
batch = asa.save_batch(batch)
print(f"Batch criado: {batch.id}")
ec = asapy.ExecutionController(sim_func=asapy.batch_simulate(batch=batch), stop_func=asapy.stop_func(metric=metric, threshold=0.001, side=side), chunk_size=3)

O método `run` do objeto `ec` é executado, passando o DataFrame `df` os com aliases. O resultado da execução é armazenado na variável `results`.

In [ ]:
results = ec.run(doe=df)

In [ ]:
results

## Recuperando um Lote de Execuções <a name="recuperando_lote"></a>

Após a conclusão do processo em lote, os usuários podem recuperá-lo para análise. Nesta seção, salvamos os aliases e os resultados das simulações para análises futuras.

In [ ]:
import os

# Create 'results' directory if it doesn't exist
if not os.path.exists('results'):
    os.makedirs('results')

# Save the DataFrames to CSV files in the 'results' directory
df.to_csv(f'./results/{simulation_name}_batch_{batch.id}_aliases.csv', index=False)
results.to_csv(f"./results/{simulation_name}_batch_{batch.id}.csv", index=False)